<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">


## Space
The analysis we did during the workshop did not take into account physical space. If neurons near each other have more similar functional properties, and neurons near to one another are more likely to be connected, this effect might be explained just by spatial factors. How big are those effects? Can they explain this shift?

There is good evidence that synapses located close to the cell body of equal size are functionally stronger than synapses which are farther from the cell body.  

This exercise is longer and more complex than the others

In [1]:
import numpy as np
import os
import caveclient
import scipy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline



<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

### We will start with recalculating the dataframe from the workshop

In [2]:
import platform
import os

platstring = platform.platform()
if ('Darwin' in platstring) or ('macOS' in platstring):
    # macOS 
    data_root = "/Volumes/Brain2023/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn' in platstring):
    # then on Code Ocean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2023/"
    
data_dir = os.path.join(data_root, 'microns_in_silico')

# you can just override this if the location of the data varies
# data_dir = '/Users/forrestc/Downloads/microns_in_silico/'

In [3]:
# we are going to load up the data and prepare the dataframe like we did 
# in class but with fewer comments

# load up the in-silico responses as a pandas dataframe from a numpy array 
resp=pd.DataFrame(np.load(os.path.join(data_dir, 'nat_resp.npy')))

# load up the csv of metadata about the 104171 units
units_df = pd.read_csv(os.path.join(data_dir, 'nat_unit.csv'))

# set the index to the be the row_idx of the units_df
resp.index = units_df['row_idx']

# if we are on code ocean, the CAVEsetup helped you make your token an environment variable
if 'amzn' in platstring:
    client= caveclient.CAVEclient('minnie65_public', auth_token=os.environ['API_SECRET'])
else:
    # otherwise if you are local, then it should be saved to a file in your harddrive 
    # that the caveclient knows where to read.
    client= caveclient.CAVEclient('minnie65_public')

# lets pull out the manual coregistered neurons
# desired_resolution describes how many nanometers you want each unit to be
# so 1000,1000,1000 gives positions in microns for x,y and z
coreg_df = client.materialize.query_table('coregistration_manual_v3', desired_resolution=[1000,1000,1000])

# lets merge these dataframes so we get the row_idx of each coregistered unit
# we merge on the corresponding columns, however scan was called something
# slightly different in one csv vs the CAVE table
coreg_in_silico=pd.merge(units_df, coreg_df, 
         left_on=['scan_session', 'scan_idx', 'unit_id'],
          right_on=['session','scan_idx', 'unit_id'])
# reset the index to make sure that we have the index
coreg_in_silico.reset_index(inplace=True)

# this will pull out the responses to the coregistered units
# by using the row_idx that was provided in the metadata
coreg_resp = resp.loc[coreg_in_silico.row_idx,:]

# now with a reduced set of units, we can calculate the Pearson correlation
# between their responses
corr_M = np.corrcoef(coreg_resp.values)




<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

However this time lets make a dataframe that contains all the correlations
but also the nucleus IDs of both sides of the correlation
and then merge in the nucleus positions so we can measure the
soma to soma distance of that correlation


In [4]:
# get an array of the nucleus IDs of each row/column of the corr_M


# get the row and column indices of the upper right triangle
# of this matrix

# use the row and column indices to get an array of nucleus IDs on each side of the correlation


# use fancy indexing to pull out the correlation values


In [5]:
# construct a dataframe using these 3 columns
# hint use a a dictionary to name the columns
# and include "copy=False" to avoid blowing up memory


In [6]:
# get the nucleus positions dataframe
# converting the positions to microns
# and using standard transform to adjust them to be flat
nuc_df = client.materialize.query_view('nucleus_detection_lookup_v1', 
                                        select_columns = ['id', 'pt_root_id', 'pt_position'],
                                        desired_resolution=[1,1,1])
from standard_transform.datasets import minnie_transform_nm
tform=minnie_transform_nm()
nuc_df['pt_position']=tform.apply(nuc_df.pt_position)
nuc_df['pt_position']=nuc_df.pt_position.apply(np.array)

In [7]:
# merge on the pre and post positions


In [1]:
# visualize the first few rows of your dataframe


In [9]:
# measure the distance between the soma of nuc1 and nuc2

# measure the distance also in x,z only.. this is along the surface of cortex

# hints: look at np.vstack, np.linalg.norm


In [11]:
# filter out distances of <2 microns


In [2]:
# using binned statistic, lets measure the avg C as a function of euclidean distance

# make up some distance bins from 2-250 microns


# use scipy.stats.binned_statistic
# to measure correlation as a function of distance



In [15]:
# what about using the cortical distance
# use the same bins




In [3]:
# make a plot of mean Correlation and std error bars a function of distance
# put both distances on same plot


In [4]:
# make a plot of how many pairs fall in each of these distance bins


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

#### Thought question

What explains these curves? 

In [19]:
# make the same plot but for connected pairs of neurons
# first lets reconstruct the dataframe from the workshop
# we need this code to work in solutions directory
# and one up..
if 'solutions' in os.getcwd():
    workshop2file = '../../workshop2/all_prf_coreg_conn_v661.pkl'
else:
    workshop2file = '../workshop2/all_prf_coreg_conn_v661.pkl'
all_syn_df = pd.read_pickle(workshop2file)

# lets merge on the pre and post-synaptic positions of these connections

# renaming the positions as pre and post depending on how we did the merge
# and drop the duplicate id columns
all_syn_dfm=all_syn_df.merge(nuc_df[['id', 'pt_position']], left_on='pre_nuc_id', right_on='id')\
.rename({'pt_position':'pre_pt_position'}, axis=1)\
.merge(nuc_df[['id', 'pt_position']], left_on='post_nuc_id', right_on='id')\
.rename({'pt_position':'post_pt_position'}, axis=1)\
.drop(['id_x', 'id_y'], axis=1)

# now lets merge in the neurons that are coregistered with responses

# we have to drop duplicates to avoid the few cells that were coregistered twice 
# being double counted
all_syn_dfm2=all_syn_dfm.merge(coreg_in_silico[['index','target_id', 'scan_session', 'scan_idx', 'field','unit_id', 'score', 'residual']],
                  left_on='pre_nuc_id', 
                  right_on='target_id')\
.merge(coreg_in_silico[['index','target_id', 'scan_session', 'scan_idx', 'field','unit_id','score', 'residual']],
                  left_on='post_nuc_id', 
                  right_on='target_id',
                  suffixes=['_pre', '_post'])\
.drop(['target_id_pre', 'target_id_post'],axis=1)\
.drop_duplicates(subset=['pre_nuc_id', 'post_nuc_id'])
all_syn_dfm2

# now use fancy indexing to pull out the correlation associated with each of these connections
all_syn_dfm2['C']=corr_M[all_syn_dfm2.index_pre, all_syn_dfm2.index_post]



In [5]:
# lets cut the dataframe down to just the columns we need
df_conn=all_syn_dfm2[['pre_nuc_id', 'post_nuc_id', 'n_syn', 'sum_size', 'C', 'pre_pt_position', 'post_pt_position']]

In [21]:
df_conn.head()

,pre_nuc_id,post_nuc_id,n_syn,sum_size,C,pre_pt_position,post_pt_position
0,265045,303145,1,1732,0.169119,"[643.0300065048364, 427.88641564546697, 884.36...","[667.8080332150954, 489.8015688177371, 890.640..."
1,299091,303145,1,7604,0.010837,"[709.1922761459178, 345.01749614037846, 895.20...","[667.8080332150954, 489.8015688177371, 890.640..."
2,292685,303145,1,23668,0.192392,"[728.1902267066858, 144.7592334831523, 840.840...","[667.8080332150954, 489.8015688177371, 890.640..."
4,256576,303145,1,9404,0.040026,"[618.4397512164188, 125.9061074383746, 881.280...","[667.8080332150954, 489.8015688177371, 890.640..."
5,222998,303145,2,25900,0.145264,"[600.9084008376958, 131.6961825586517, 919.880...","[667.8080332150954, 489.8015688177371, 890.640..."


In [6]:
# calculate the intersoma distance


In [26]:
# filter out the soma distances of <2 microns
# to discount the double roi cells


In [27]:
# use scipy.stats.binned_statistic
# to measure correlation as a function of distance for connected


In [7]:
#  plot both curves for connected and all paris on top of one another
# make a plot of mean Correlation and std error bars a function of distance


In [8]:
#  plot both curves for connected and all paris on top of one another
# make a plot of mean Correlation and std error bars a function of distance


In [9]:
# plot how many connected pairs are in each distance bin


In [10]:
# plot how many are connected in a cortical distance bin


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

#### Thought questions

What explains the difference between this curve and the overall distribution of pairs of recorded ROIs? 

Does this curve match your expectation for what cortical connectivity should look like?

If space explains a lot of the differences between connected and unconnected pairs,
what does that mean?

How does it affect your interpretation of the effects?

Are there spatial effects that go beyond just soma to soma distance that would be important to control for in order to interpret a finding as being evidence for a particular mechanism?

#### Extensions/Project Ideas

Can you make a model which resamples the all pairs to match the spatial distributions found in the connected dataset?  Are the results significant compared to that null model?

Does this explain the variation seen in the single cell effects.. that some cells have closer and farther away targets in the brain?

</div>